# Machine Learning Trading Bot

In this Challenge, you’ll assume the role of a financial advisor at one of the top five financial advisory firms in the world. Your firm constantly competes with the other major firms to manage and automatically trade assets in a highly dynamic environment. In recent years, your firm has heavily profited by using computer algorithms that can buy and sell faster than human traders.

The speed of these transactions gave your firm a competitive advantage early on. But, people still need to specifically program these systems, which limits their ability to adapt to new data. You’re thus planning to improve the existing algorithmic trading systems and maintain the firm’s competitive advantage in the market. To do so, you’ll enhance the existing trading signals with machine learning algorithms that can adapt to new data.

## Instructions:

Use the starter code file to complete the steps that the instructions outline. The steps for this Challenge are divided into the following sections:

* Establish a Baseline Performance

* Tune the Baseline Trading Algorithm

* Evaluate a New Machine Learning Classifier

* Create an Evaluation Report

#### Establish a Baseline Performance

In this section, you’ll run the provided starter code to establish a baseline performance for the trading algorithm. To do so, complete the following steps.

Open the Jupyter notebook. Restart the kernel, run the provided cells that correspond with the first three steps, and then proceed to step four. 

1. Import the OHLCV dataset into a Pandas DataFrame.

2. Generate trading signals using short- and long-window SMA values. 

3. Split the data into training and testing datasets.

4. Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

5. Review the classification report associated with the `SVC` model predictions. 

6. Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

7. Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

8. Write your conclusions about the performance of the baseline trading algorithm in the `README.md` file that’s associated with your GitHub repository. Support your findings by using the PNG image that you saved in the previous step.

#### Tune the Baseline Trading Algorithm

In this section, you’ll tune, or adjust, the model’s input features to find the parameters that result in the best trading outcomes. (You’ll choose the best by comparing the cumulative products of the strategy returns.) To do so, complete the following steps:

1. Tune the training algorithm by adjusting the size of the training dataset. To do so, slice your data into different periods. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. Answer the following question: What impact resulted from increasing or decreasing the training window?

> **Hint** To adjust the size of the training dataset, you can use a different `DateOffset` value&mdash;for example, six months. Be aware that changing the size of the training dataset also affects the size of the testing dataset.

2. Tune the trading algorithm by adjusting the SMA input features. Adjust one or both of the windows for the algorithm. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. Answer the following question: What impact resulted from increasing or decreasing either or both of the SMA windows?

3. Choose the set of parameters that best improved the trading algorithm returns. Save a PNG image of the cumulative product of the actual returns vs. the strategy returns, and document your conclusion in your `README.md` file.

#### Evaluate a New Machine Learning Classifier

In this section, you’ll use the original parameters that the starter code provided. But, you’ll apply them to the performance of a second machine learning model. To do so, complete the following steps:

1. Import a new classifier, such as `AdaBoost`, `DecisionlogClassifier`, or `LogisticRegression`. (For the full list of classifiers, refer to the [Supervised learning page](https://scikit-learn.org/stable/supervised_learning.html) in the scikit-learn documentation.)

2. Using the original training data as the baseline model, fit another model with the new classifier.

3. Backtest the new model to evaluate its performance. Save a PNG image of the cumulative product of the actual returns vs. the strategy returns for this updated trading algorithm, and write your conclusions in your `README.md` file. Answer the following questions: Did this new model perform better or worse than the provided baseline model? Did this new model perform better or worse than your tuned trading algorithm?

#### Create an Evaluation Report

In the previous sections, you updated your `README.md` file with your conclusions. To accomplish this section, you need to add a summary evaluation report at the end of the `README.md` file. For this report, express your final conclusions and analysis. Support your findings by using the PNG images that you created.


In [ ]:
# Imports
import pandas as pd
from pandas.tseries.offsets import DateOffset
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm, tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print('Loaded Libraries!')

---

## Establish a Baseline Performance

In this section, you’ll run the provided starter code to establish a baseline performance for the trading algorithm. To do so, complete the following steps.

Open the Jupyter notebook. Restart the kernel, run the provided cells that correspond with the first three steps, and then proceed to step four. 


### Step 1: Import the OHLCV dataset into a Pandas DataFrame.

In [ ]:
# Import the OHLCV dataset into a Pandas Dataframe
ohlcv_df = pd.read_csv(
    Path("./Resources/emerging_markets_ohlcv.csv"), 
    index_col='date', 
    infer_datetime_format=True, 
    parse_dates=True
)

# Review the DataFrame
ohlcv_df.head()

In [ ]:
# Filter the date index and close columns
signals_df = ohlcv_df.loc[:, ["close"]]

# Use the pct_change function to generate returns from close prices
signals_df["Actual Returns"] = signals_df["close"].pct_change()

# Drop all NaN values from the DataFrame
signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

## Step 2: Generate trading signals using short- and long-window SMA values. 

In [ ]:
# Set the short window and long window
short_window = 4
long_window = 100

# Generate the fast and slow simple moving averages (4 and 100 days, respectively)
signals_df['SMA_Fast'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA_Slow'] = signals_df['close'].rolling(window=long_window).mean()

signals_df = signals_df.dropna()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
# Initialize the new Signal column
signals_df['Signal'] = 0.0

# When Actual Returns are greater than or equal to 0, generate signal to buy stock long
signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1

# When Actual Returns are less than 0, generate signal to sell stock short
signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = -1

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
signals_df['Signal'].value_counts()

In [ ]:
# Calculate the strategy returns and add them to the signals_df DataFrame
signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

# Review the DataFrame
display(signals_df.head())
display(signals_df.tail())

In [ ]:
# Plot Strategy Returns to examine performance
(1 + signals_df['Strategy Returns']).cumprod().plot()

### Step 3: Split the data into training and testing datasets.

In [ ]:
# Assign a copy of the sma_fast and sma_slow columns to a features DataFrame called X
X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna().copy()

# Review the DataFrame
display(X.head())
display(X.tail())

In [ ]:
# Create the target set selecting the Signal column and assiging it to y
y = signals_df['Signal']

# Review the value counts
y.value_counts()

In [ ]:
# Select the start of the training period
training_begin = X.index.min()

# Display the training begin date
print(training_begin)

In [ ]:
# Select the ending period for the training data with an offset of 3 months
training_end = X.index.min() + DateOffset(months=3)

# Display the training end date
print(training_end)

In [ ]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

# Review the X_train DataFrame
X_train.head()


In [ ]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

# Review the X_test DataFrame
X_test.tail()

In [ ]:
# Scale the features DataFrames
# Create a StandardScaler instance
scaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Step 4: Use the `SVC` classifier model from SKLearn's support vector machine (SVM) learning method to fit the training data and make predictions based on the testing data. Review the predictions.

In [ ]:
# From SVM, instantiate SVC classifier model instance
svm_model = svm.SVC()
 
# Fit the model to the data using the training data
svm_model = svm_model.fit(X_train_scaled, y_train)
 
# Use the testing data to make the model predictions
svm_pred = svm_model.predict(X_test_scaled)

# Review the model's predicted values
svm_pred[:5]


### Step 5: Review the classification report associated with the `SVC` model predictions. 

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report = classification_report(y_test, svm_pred)

# Print the classification report
print(svm_testing_report)


### Step 6: Create a predictions DataFrame that contains columns for “Predicted” values, “Actual Returns”, and “Strategy Returns”.

In [ ]:
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index)

# Add the SVM model predictions to the DataFrame
predictions_df['Predicted'] = svm_pred

# Add the actual returns to the DataFrame
predictions_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
predictions_df['Strategy Returns'] = (predictions_df['Actual Returns'] * predictions_df['Predicted'])

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail())

### Step 7: Create a cumulative return plot that shows the actual returns vs. the strategy returns. Save a PNG image of this plot. This will serve as a baseline against which to compare the effects of tuning the trading algorithm.

In [ ]:
# Plot the actual returns versus the strategy returns
(1 + predictions_df[['Actual Returns', 'Strategy Returns']]).cumprod().plot()
plt.savefig('./img/svc_plot.png', bbox_inches='tight')

---

## Tune the Baseline Trading Algorithm

In this section, you’ll tune, or adjust, the model’s input features to find the parameters that result in the best trading outcomes. You’ll choose the best by comparing the cumulative products of the strategy returns.

### Step 1: Tune the training algorithm by adjusting the size of the training dataset. 

To do so, slice your data into different periods. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. 

Answer the following question: What impact resulted from increasing or decreasing the training window?

### Step 2: Tune the trading algorithm by adjusting the SMA input features. 

Adjust one or both of the windows for the algorithm. Rerun the notebook with the updated parameters, and record the results in your `README.md` file. 

Answer the following question: What impact resulted from increasing or decreasing either or both of the SMA windows?

### Step 3: Choose the set of parameters that best improved the trading algorithm returns. 

Save a PNG image of the cumulative product of the actual returns vs. the strategy returns, and document your conclusion in your `README.md` file.

In [ ]:
## TUNE STEP 1 ##

# Select the start of the training period
tuned_begin = X.index.min()

# Select the ending period for the training data with an offset of 6 months
tuned_end = X.index.min() + DateOffset(months=6)

# Generate the X_train and y_train DataFrames
X_ttrain = X.loc[tuned_begin:tuned_end]
y_ttrain = y.loc[tuned_begin:tuned_end]

# Generate the X_test and y_test DataFrames
X_ttest = X.loc[tuned_end+DateOffset(hours=1):]
y_ttest = y.loc[tuned_end+DateOffset(hours=1):]

# Create a StandardScaler instance
tscaler = StandardScaler()

# Apply the scaler model to fit the X-train data
X_tscaler = tscaler.fit(X_ttrain)

# Transform the X_train and X_test DataFrames using the X_scaler
X_ttrain_scaled = X_tscaler.transform(X_ttrain)
X_ttest_scaled = X_tscaler.transform(X_ttest)

# From SVM, instantiate SVC classifier model instance
tsvm_model = svm.SVC()
 
# Fit the model to the data using the training data
tsvm_model = tsvm_model.fit(X_ttrain_scaled, y_ttrain)
 
# Use the testing data to make the model predictions
tsvm_pred = tsvm_model.predict(X_ttest_scaled)

# Use a classification report to evaluate the model using the predictions and testing data
tsvm_testing_report = classification_report(y_ttest, tsvm_pred)

# Print the classification report
print(tsvm_testing_report)

In [ ]:
# Create a predictions DataFrame
tuned_pred_df = pd.DataFrame(index=X_ttest.index)

# Add the SVM model predictions to the DataFrame
tuned_pred_df['Predicted'] = tsvm_pred

# Add the actual returns to the DataFrame
tuned_pred_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
tuned_pred_df['Strategy Returns'] = (tuned_pred_df['Actual Returns'] * tuned_pred_df['Predicted'])

# Plot the actual returns versus the strategy returns
(1 + tuned_pred_df[['Actual Returns', 'Strategy Returns']]).cumprod().plot()
plt.savefig('./img/svc_plot2.png', bbox_inches='tight')

In [ ]:
## TUNE STEP 2 ##

# Copy original data frame
tuned_sig_df = signals_df.copy()

# Tuned windows
tuned_swin = 1
tuned_lwin = 25

# Tuned fast and slow simple moving averages
tuned_sig_df['SMA_Fast'] = tuned_sig_df['close'].rolling(window=tuned_swin).mean()
tuned_sig_df['SMA_Slow'] = tuned_sig_df['close'].rolling(window=tuned_lwin).mean()

tuned_sig_df = tuned_sig_df.dropna()

X2 = tuned_sig_df[['SMA_Fast', 'SMA_Slow']].shift().dropna().copy()
y2 = tuned_sig_df['Signal']

# Select the start of the training period
tuned_begin = X2.index.min()

# Select the ending period for the training data with an offset of 6 months
tuned_end = X2.index.min() + DateOffset(months=6)

# Generate the X_train and y_train DataFrames
X2_train = X2.loc[tuned_begin:tuned_end]
y2_train = y2.loc[tuned_begin:tuned_end]

# Generate the X_test and y_test DataFrames
X2_test = X2.loc[tuned_end+DateOffset(hours=1):]
y2_test = y2.loc[tuned_end+DateOffset(hours=1):]

# Create a StandardScaler instance
scaler2 = StandardScaler()

# Apply the scaler model to fit the X-train data
X2_scaler = scaler2.fit(X2_train)

# Transform the X_train and X_test DataFrames using the X_scaler
X2_train_scaled = X2_scaler.transform(X2_train)
X2_test_scaled = X2_scaler.transform(X2_test)

# From SVM, instantiate SVC classifier model instance
svm_model2 = svm.SVC()
 
# Fit the model to the data using the training data
svm_model2 = svm_model2.fit(X2_train_scaled, y2_train)
 
# Use the testing data to make the model predictions
svm_pred2 = svm_model2.predict(X2_test_scaled)

# Use a classification report to evaluate the model using the predictions and testing data
svm_testing_report2 = classification_report(y2_test, svm_pred2)

# Print the classification report
print(svm_testing_report2)

In [ ]:
# Create a predictions DataFrame
tuned_pred_df2 = pd.DataFrame(index=X2_test.index)

# Add the SVM model predictions to the DataFrame
tuned_pred_df2['Predicted'] = svm_pred2

# Add the actual returns to the DataFrame
tuned_pred_df2['Actual Returns'] = tuned_sig_df['Actual Returns']

# Add the strategy returns to the DataFrame
tuned_pred_df2['Strategy Returns'] = (tuned_pred_df2['Actual Returns'] * tuned_pred_df2['Predicted'])

# Plot the actual returns versus the strategy returns
(1 + tuned_pred_df2[['Actual Returns', 'Strategy Returns']]).cumprod().plot()
plt.savefig('./img/svc_plot3.png', bbox_inches='tight')

---

## Evaluate a New Machine Learning Classifier

In this section, you’ll use the original parameters that the starter code provided. But, you’ll apply them to the performance of a second machine learning model. 

### Step 1:  Import a new classifier, such as `AdaBoost`, `DecisionTreeClassifier`, or `LogisticRegression`. (For the full list of classifiers, refer to the [Supervised learning page](https://scikit-learn.org/stable/supervised_learning.html) in the scikit-learn documentation.)

In [ ]:
# Initiate the model instance
tree_model = tree.DecisionTreeClassifier()

### Step 2: Using the original training data as the baseline model, fit another model with the new classifier.

In [ ]:
# Fit the model using the training data
tree_model = tree_model.fit(X_train_scaled, y_train)

# Use the testing dataset to generate the predictions for the new model
tree_pred = tree_model.predict(X_test_scaled)

# Review the model's predicted values
tree_pred[:5]

### Step 3: Backtest the new model to evaluate its performance. 

Save a PNG image of the cumulative product of the actual returns vs. the strategy returns for this updated trading algorithm, and write your conclusions in your `README.md` file. 

Answer the following questions: 
Did this new model perform better or worse than the provided baseline model? 
Did this new model perform better or worse than your tuned trading algorithm?

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
tree_test_rep = classification_report(y_test, tree_pred)

# Print the classification report
print(tree_test_rep)

In [ ]:
# Create a predictions DataFrame
tree_pred_df = pd.DataFrame(index=X_test.index)

# Add the tree model predictions to the DataFrame
tree_pred_df['Predicted'] = tree_pred

# Add the actual returns to the DataFrame
tree_pred_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
tree_pred_df['Strategy Returns'] = (tree_pred_df['Actual Returns'] * tree_pred_df['Predicted'])

# Review the DataFrame
tree_pred_df.head()

In [ ]:
# Plot the actual returns versus the strategy returns
(1 + tree_pred_df[['Actual Returns', 'Strategy Returns']]).cumprod().plot()
plt.savefig('./img/tree_plot.png', bbox_inches='tight')

In [ ]:
# Decided to run a Logistic Regression model for further comparison
# Initiate the model instance
log_model = LogisticRegression(random_state=1)  #solver='lbfgs', 

# Fit the model using the training data
log_model = log_model.fit(X_train_scaled, y_train)

# Use the test data to generate the predictions for the new model
log_pred = log_model.predict(X_test_scaled)

# Review the model's predicted values
log_pred[:5]

In [ ]:
# Display the accuracy score for the test dataset.
accuracy_score(y_test, log_pred)

In [ ]:
# Use a classification report to evaluate the model using the predictions and testing data
log_test_rep = classification_report(y_test, log_pred)

# Print the classification report
print(log_test_rep)

In [ ]:
# Use the trained model to predict the trading signals for the testing data.
log_sig_pred = log_model.predict(X_test_scaled)

# Create a predictions DataFrame
log_pred_df = pd.DataFrame(index=X_test.index)

# Add the tree model predictions to the DataFrame
log_pred_df['Predicted'] = log_sig_pred

# Add the actual returns to the DataFrame
log_pred_df['Actual Returns'] = signals_df['Actual Returns']

# Add the strategy returns to the DataFrame
log_pred_df['Strategy Returns'] = (log_pred_df['Actual Returns'] * log_pred_df['Predicted'])

# Review the DataFrame
log_pred_df.head()

In [ ]:
# Plot the actual returns versus the strategy returns
(1 + log_pred_df[['Actual Returns', 'Strategy Returns']]).cumprod().plot()
plt.savefig('./img/log_plot.png', bbox_inches='tight')